In [24]:
import pyspark as ps

spark = (ps.sql.SparkSession.builder 
        .master("local[4]") 
        .appName("nathanscope") 
        .getOrCreate()
        )

In [25]:
sc = spark.sparkContext

# Abstract

# Create Spark Data Frames
- Auth: Authentication events between networked computer
- Proc: Start and Stop Times for Proccesses run on machines from AV Software
- Flow: Duration, Byte and Packet Count for Data Moving Between Machines
- DNS: DNS Entries and Lookup Resolution(Not used in IP Literal Transactions)

## Define Casting Functions for RDDS

In [26]:
def auth_casting_function(row):
    (time, scr_user, dst_user, src_comp, dst_comp, auth_type,logon_type,auth_orientation,success) = row
    if(time=='?'):
        time = 666999666
    return (int(time),scr_user, dst_user, src_comp, dst_comp, auth_type,logon_type,auth_orientation,success)

In [27]:
def proc_casting_function(row):
    (time,user_domain,comp,proc_name,start) = row
    if(time=='?'):
        time = 666999666
    return (int(time),user_domain,comp,proc_name,start)

In [28]:
def flow_casting_function(row):
    (time, duration,src_comp,src_port,dst_comp,dst_port,protocol,pk_count,byte_count) = row
    if(time=='?'):
        time = 666999666
    return (int(time),int(duration),src_comp,src_port,dst_comp,dst_port,protocol,int(pk_count),int(byte_count))

In [29]:
def DNS_casting_function(row):
    (time, src_comp,comp_resolved) = row
    if(time=='?'):
        time = 666999666
    return (int(time),src_comp,comp_resolved)

## Define Schema for DataFrames

In [30]:
# import the many data types
from pyspark.sql.types import *

# create a schema of your own
auth_schema = StructType( [
    StructField('time',     LongType(), True),
    StructField('src_user_domain',   StringType(),  True),
    StructField('dst_user_domain',   StringType(),  True),
    StructField('src_comp',   StringType(),  True),
    StructField('dst_comp',   StringType(),  True),
    StructField('auth_type',  StringType(), True),
    StructField('logon_type',  StringType(),  True),
    StructField('auth_orientation',  StringType(),  True),
    StructField('Success',  StringType(),  True)] )

In [31]:
proc_schema = StructType( [
    StructField('time',     LongType(), True),
    StructField('user_domain',   StringType(),  True),
    StructField('comp',   StringType(),  True),
    StructField('proc_name',   StringType(),  True),
    StructField('start',   StringType(),  True),
    ] )

In [32]:
flow_schema = StructType( [
    StructField('time',     LongType(), True),
    StructField('duration',     LongType(), True),
    StructField('src_comp',   StringType(),  True),
    StructField('src_port',   StringType(),  True),
    StructField('dst_comp',   StringType(),  True),
    StructField('dst_port',   StringType(),  True),
    StructField('protocol',  StringType(), True),
    StructField('pk_count',  LongType(),  True),
    StructField('byte_count',  LongType(),  True)] )

In [33]:
DNS_schema = StructType( [
    StructField('time',     LongType(), True),
    StructField('src_comp',   StringType(),  True),
    StructField('comp_resolved',   StringType(),  True),
    
    ] )

## Create RDDS from Text Input

In [34]:
rdd_auth = (sc.textFile('data/auth.txt')
            .map(lambda rowstr : rowstr.split(","))
            .map(auth_casting_function)
           )

In [35]:
rdd_proc = (sc.textFile('data/proc.txt')
            .map(lambda rowstr : rowstr.split(","))
            .map(proc_casting_function)
           )

In [36]:
rdd_flow = (sc.textFile('data/flows.txt')
            .map(lambda rowstr : rowstr.split(","))
            .map(flow_casting_function)
           )

In [37]:
rdd_DNS = (sc.textFile('data/dns.txt')
            .map(lambda rowstr : rowstr.split(","))
            .map(DNS_casting_function)
           )

In [38]:
rdd_sample = (sc.textFile('data/auth_sample.txt')
            .map(lambda rowstr : rowstr.split(","))
            .map(auth_casting_function)
           )

In [39]:
rdd_sample.filter(lambda x: x[3] == 'C988').collect()

[(1,
  'C101$@DOM1',
  'C101$@DOM1',
  'C988',
  'C988',
  '?',
  'Network',
  'LogOff',
  'Success')]

# Sample First 30 Day
## filter by items in firtst day and Save to Local Files

In [40]:
rdd_auth.filter(lambda x: x[0] < 2592000).saveAsPickleFile('Data/Day30/auth')

In [41]:
rdd_flow.filter(lambda x: x[0] < 2592000).saveAsPickleFile('Data/Day30/flow')

In [42]:
rdd_proc.filter(lambda x: x[0] < 2592000).saveAsPickleFile('Data/Day30/proc')

In [43]:
rdd_DNS.filter(lambda x: x[0] < 2592000).saveAsPickleFile('Data/Day30/DNS')

In [44]:
DNS_1_len = DNS_df_1.count()

NameError: name 'DNS_df_1' is not defined

In [ ]:
DNS_1_len

schema = StructType( [
    StructField('id',     IntegerType(), True),
    StructField('date',   StringType(),  True),
    StructField('store',  IntegerType(), True),
    StructField('state',  StringType(),  True),
    StructField('product',IntegerType(), True),
    StructField('amount', FloatType(),   True) ] )

len_auth = rdd_auth.count()

len_auth